<a href="https://colab.research.google.com/github/Alla-ud-din/Classes-Q3-Q4/blob/langgraph/LangGraph/module-2/6.2_chatbot_external_memory_with_postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot with message summarization & external DB memory

## Review

We've covered how to customize graph state schema and reducer.

We've also shown a number of tricks for trimming or filtering messages in graph state.

We've used these concepts in a Chatbot with memory that produces a running summary of the conversation.

## Goals

But, what if we want our Chatbot to have memory that persists indefinitely?

Now, we'll introduce some more advanced checkpointers that support external databases.

Here, we'll show how to use [Postgres as a checkpointer](https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/)

In [2]:
%%capture --no-stderr
%pip install -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai


In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
import os
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

# Use sync connection¶
This sets up a synchronous connection to the database.

Synchronous connections execute operations in a blocking manner, meaning each operation waits for completion before moving to the next one. The DB_URI is the database connection URI, with the protocol used for connecting to a PostgreSQL database, authentication, and host where database is running. The connection_kwargs dictionary defines additional parameters for the database connection.

In [5]:
from google.colab import userdata
DB_URI = userdata.get('DB_URI')

In [6]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Connection pool for efficient database access
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}

# Create a persistent connection pool
pool = ConnectionPool(conninfo=DB_URI, max_size=20, kwargs=connection_kwargs)

# Initialize PostgresSaver checkpointer
checkpointer = PostgresSaver(pool)
checkpointer.setup()  # Ensure database tables are set up


Let's re-define our chatbot.

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =  GEMINI_API_KEY)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END

Now, we just re-compile with our postgres checkpointer.

In [8]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Redefine workflow
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile the workflow with PostgreSQL checkpointer
graph = workflow.compile(checkpointer=checkpointer)


Now, we can invoke the graph several times.

In [10]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="hi! I'm Wania")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 2
================================== Ai Message ==================================

Hi Wania!  It's nice to meet you. How can I help you today?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Wania", additional_kwargs={}, response_metadata={}, id='45866f7c-a271-478f-a209-b1eb39bff715'), AIMessage(content="Hi Wania!  It's nice to meet you. How can I help you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c3ca8caf-b414-4cff-9a8f-62e822c2292f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 21, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efca658-adcd-610f-8001-a6d86f30483b'}}, metadata={'step': 1, 'source': 'loop', 'writes': {'conversation': {'messages': AIMessage(content="Hi Wania!  It's nice to meet you. How can I help you today?\n", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [], 'prompt_feedback': {'block_reason': 0, 'safety_ratin

In [11]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="I like painting pictures.")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 4
================================== Ai Message ==================================

That's wonderful!  What kind of pictures do you like to paint?  Do you have a favorite medium (oil, acrylic, watercolor, etc.)?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Wania", additional_kwargs={}, response_metadata={}, id='45866f7c-a271-478f-a209-b1eb39bff715'), AIMessage(content="Hi Wania!  It's nice to meet you. How can I help you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c3ca8caf-b414-4cff-9a8f-62e822c2292f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 21, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='I like painting pictures.', additional_kwargs={}, response_metadata={}, id='7269d870-703a-4b9b-b588-a7ab7b2fb52a'), AIMessage(content="That's wonderful!  What kind of pictures do you like to paint?  Do you have a favorite medium (oil, acrylic, watercolor, etc.)?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [

In [12]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="What's my name and what is my hobby?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 6
================================== Ai Message ==================================

Your name is Wania, and your hobby is painting pictures.


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Wania", additional_kwargs={}, response_metadata={}, id='45866f7c-a271-478f-a209-b1eb39bff715'), AIMessage(content="Hi Wania!  It's nice to meet you. How can I help you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c3ca8caf-b414-4cff-9a8f-62e822c2292f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 21, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='I like painting pictures.', additional_kwargs={}, response_metadata={}, id='7269d870-703a-4b9b-b588-a7ab7b2fb52a'), AIMessage(content="That's wonderful!  What kind of pictures do you like to paint?  Do you have a favorite medium (oil, acrylic, watercolor, etc.)?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [

In [10]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="Can you describe about abstract paintings?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: The conversation began with introductions, where I learned the user's name is Wania and her hobby is painting.  She then asked about abstract painting, and I provided a detailed description of the art style, including its characteristics, different movements within it, and its emphasis on form, color, and emotional expression over representational accuracy.

Message count: 6
================================== Ai Message ==================================

Abstract painting is a form of art that doesn't aim to represent the visible world realistically. Instead, it uses shapes, colors, lines, and textures to express ideas, emotions, or explore the visual elements themselves.  It's about the *feeling* or *concept* conveyed, rather than a literal depiction of something.

Key characteristics include:

* **Non-representational:** Unlike realistic art, there aren't recognizable objects or scenes.  It's not about "what" is depicted, but "how" it's depicted.

* **Emphasis on visu

StateSnapshot(values={'messages': [HumanMessage(content='Can you describe about abstract paintings?', additional_kwargs={}, response_metadata={}, id='0a624a56-cb73-4c3f-a414-3d861aef5836'), AIMessage(content='Abstract painting is a genre of art that doesn\'t attempt to represent an accurate depiction of visual reality but instead uses shapes, colors, forms, and gestural marks to achieve its effect.  It\'s less about representing something recognizable and more about expressing feelings, ideas, or exploring the elements of art themselves.\n\nHere are some key characteristics of abstract painting:\n\n* **Non-representational:**  Unlike realistic or representational art, abstract art doesn\'t aim to portray objects or scenes from the real world in a recognizable way.\n\n* **Emphasis on form and color:**  The focus is often on the interplay of shapes, lines, colors, and textures, rather than subject matter.  Color can be used expressively, symbolically, or purely for its aesthetic qualitie

In [1]:
# Retrieve state using thread ID
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config).values.get('summary', "")
graph_state

NameError: name 'graph' is not defined

In [ ]:
pool.close()

### Persisting state

Using database like Postgres means state is persisted!

For example, we can re-start the notebook kernel and see that we can still load from Postgres DB on disk.
